# Angular correlation of bright object in LSST

- Last update 2024-07-31
- 

In [ ]:
import astropy.units as u
import numpy as np 
import pandas as pd
pd.set_option("display.max_columns", None)
from astropy.time import Time

import scipy.stats

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus


import lsst.geom as geom
import pickle
from astropy.stats import SigmaClip

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS


import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

import scipy
from scipy.optimize import curve_fit

In [ ]:
import lsst
import lsst.sphgeom
from  lsst.geom import SpherePoint, Angle

#spherePointCenter = lsst.geom.SpherePoint(ra1*lsst.geom.degrees, dec1*lsst.geom.degrees)
#spt = lsst.geom.SpherePoint(ra2*lsst.geom.degrees, dec2*lsst.geom.degrees)
#ang = spherePointCenter.separation(spt)
#ang_arcsec = ang.asArcseconds()

In [ ]:
def calculateSeparation(ra0,dec0,ra,dec):
    spt0 = lsst.geom.SpherePoint(ra0*lsst.geom.degrees, dec0*lsst.geom.degrees)
    spt = lsst.geom.SpherePoint(ra*lsst.geom.degrees, dec*lsst.geom.degrees)
    ang = spt0.separation(spt)
    ang_arcsec = ang.asArcseconds()
    return ang_arcsec
 

In [ ]:
from lsst.geom import Angle 
import lsst
func_degToRad = lambda x : Angle(x,lsst.geom.degrees).asRadians()
func_degToArcsec = lambda x : Angle(x,lsst.geom.degrees).asArcseconds()

In [ ]:
df = pd.read_csv("list_of_isolatedobjects.csv",index_col=0)

In [ ]:
df.head()

## Calculation of angle correlation

In [ ]:
all_df = []
for index, row in df.iterrows():
    ra0 = row['ra']
    dec0 = row['dec']
    df1 = df.copy(deep=True)
    df1 = df1.iloc[index+1:] 
    df1["sep"]=df1.apply(lambda x : calculateSeparation(ra0,dec0,x['ra'],x['dec']),axis=1)
    all_df.append(df1)

## Separation angle distribution

In [ ]:
all_sep = []
for df1 in all_df:
    if len(df1)>0:
        list_of_sep = df1["sep"].values
        all_sep.append(list_of_sep)
all_sep = np.concatenate(all_sep, axis=0)

In [ ]:
image_size = 0.1*4000.

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(12,4))
ax1,ax2=axs
ax1.hist(all_sep,bins=50,histtype="step",color="b",lw=2);
image_size = 0.1*4000.
ax1.axvline(image_size,color="k")
ax1.set_xlabel("$\\theta$ (arcsec)")
ax1.set_title("Bright source correlation ")

all_sep_pix = all_sep/0.1
ax2.hist(all_sep_pix,bins=50,histtype="step",color="b",lw=2);
image_size = 4000.
ax2.axvline(image_size,color="k")
ax2.set_xlabel("$\\theta$ (pixel)")
ax2.set_title("Bright source correlation ")


## Save 

In [ ]:
all_pairs = []
for index, row in df.iterrows():
    idx0 = int(row["idx_obj"] )	
    ra0 = row['ra']
    dec0 = row['dec']
    G_R0 = row['G_R']
    Z_Y0 = row['Z_Y']
    df1 = all_df[index]
    for index1, row1 in df1.iterrows():
        idx1 = int(row1['idx_obj']) 	
        ra1 = row1['ra']
        dec1 = row1['dec']
        G_R1 = row1['G_R']
        Z_Y1 = row1['Z_Y']
        sep = row1['sep']
        all_pairs.append(np.array([idx0,idx1,G_R0,G_R1,Z_Y0,Z_Y1,sep]))
        
all_pairs = np.vstack(all_pairs)    

In [ ]:
all_pairs 

In [ ]:
df_pairs = pd.DataFrame(all_pairs, columns=["objidx1","objidx2","G1_R1","G2_R2","Z1_Y1","Z2_Y2","sep"] )

In [ ]:
df_pairs.sort_values("sep",axis=0, ascending=True, inplace=True)

In [ ]:
df_pairs

In [ ]:
df_pairs.to_csv("all_pairs.csv")